# Import Libraries

In [1]:
from swarmintelligence import GreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = {
    'maxIteration': 100,
    'wolfSize': 25
}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = GreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7183902321058
	 AirplaneF16.tiff | Threshold =  3
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.8020017977378
2024.7871768318307
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8020017977378
2024.805475833105
2024.8375098105803
2024.7871768318307
2024.8020017977378
202

3208.5991222326434
3254.60318711213
3254.489312979244
Mean Fitness = 3253.022897484502
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.02713277089
	 Mandrill.tiff | Threshold =  3
1639.4542198585186
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.463360998692
1639.463360998692
1639.466171099567
1639.466171099567
1552.9797303408645
1639.466171099567
1639.466171099567
1639.466171099567
1639.4633

4217.976225406367
4217.983460413122
4217.983460413122
4202.420426145112
4217.983460413122
4218.037279885386
4217.976225406367
4218.034079186658
4217.92624195069
Mean Fitness = 4217.44908251637


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[113, 173]",1.514706,1948.718390,2270.640518,47.651238,14.569320,0.794411,0.940220,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.0956859588623047, 1.3314735889434814, 1.568...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2270....","[47.651238369936166, 47.651238369936166, 47.65...","[14.569319775617668, 14.569319775617668, 14.56...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,otsu,max,"[93, 145, 191]",1.534987,2024.822703,1769.628706,42.066500,15.652170,0.778295,0.953305,"[[93, 255, 255, 191, 191, 191, 255, 191, 255, ...","[1.3403215408325195, 1.6054017543792725, 1.550...","[2024.8375098105803, 2024.8375098105803, 2024....","[1774.3395538330078, 1774.3395538330078, 1774....","[42.12291008267363, 42.12291008267363, 42.1229...","[15.640436268576428, 15.640436268576428, 15.64...","[0.77934507049225, 0.77934507049225, 0.7793450...","[0.9532349424759257, 0.9532349424759257, 0.953..."
2,AirplaneF16.tiff,4,otsu,max,"[84, 128, 172, 203]",1.678151,2069.968634,1095.883412,33.103697,17.733385,0.732406,0.969391,"[[84, 203, 203, 203, 203, 203, 203, 203, 203, ...","[1.3316068649291992, 1.609400987625122, 1.5424...","[2070.066700775117, 2070.0794736113085, 2069.9...","[1092.5622863769531, 1094.045597076416, 1091.3...","[33.05392996871859, 33.07636009412789, 33.0351...","[17.746341556293658, 17.74044938188586, 17.751...","[0.7331993432994068, 0.7326687134455045, 0.732...","[0.9694152397220919, 0.9693723689797689, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[68, 107, 143, 180, 205]",1.648633,2095.748100,909.317179,30.154345,18.543959,0.745217,0.975428,"[[68, 205, 205, 205, 180, 205, 205, 205, 205, ...","[1.9274382591247559, 1.559429407119751, 1.8354...","[2095.775514729944, 2096.1057111326204, 2095.1...","[901.7910575866699, 905.9262161254883, 911.230...","[30.02983612320703, 30.09860820910974, 30.1865...","[18.57974436462029, 18.559875332070355, 18.534...","[0.7459399990966467, 0.7453189552872135, 0.745...","[0.9755892796900518, 0.9753885406059017, 0.975..."
4,Lena.png,2,otsu,max,"[93, 151]",1.541058,1961.821090,2813.545990,53.042869,13.638263,0.612265,0.883327,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.6014156341552734, 1.6075611114501953, 1.577...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2813.56...","[53.04306806744237, 53.04306806744237, 53.0430...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.612...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,otsu,max,"[81, 127, 171]",1.715113,2128.298661,1250.274057,35.359209,17.160754,0.706275,0.933639,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.670408010482788, 1.7029683589935303, 1.6064...","[2128.284181016679, 2128.3048672466566, 2128.2...","[1252.4491691589355, 1249.3418655395508, 1252....","[35.389958592218434, 35.346030407098766, 35.38...","[17.153202518789254, 17.16399067408044, 17.153...","[0.7072313681226016, 0.7058656377663244, 0.707...","[0.9340987864621145, 0.9334425202424386, 0.934..."
6,Lena.png,4,otsu,max,"[75, 114, 145, 180]",1.616644,2191.869825,792.449944,28.150348,19.141171,0.753871,0.954191,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[1.5748703479766846, 1.6164398193359375, 1.668...","[2191.874720070565, 2191.8485530059365, 2191.8...","[795.6595458984375, 791.031623840332, 795.6595...","[28.2074377762043, 28.12528442238997, 28.20743...","[19.12353083289055, 19.148865148104118, 19.123...","[0.7534048142269026, 0.7540597418757801, 0.753...","[0.9539824804728553, 0.9549007300695751, 0.953..."
7,Lena.png,5,otsu,max,"[70, 103, 130, 155, 185]",1.721253,2217.139341

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GWO/GWO_otsu_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GWO/GWO_otsu_dict.pkl', 'wb'))